**-- Here we retriving the basic data for Nifty 500 stocks, sorting on the basis of Market_cap to segrigate Large, Mid and Samll stocks. You can either use this Nifty 500 for ur analysis or can upoad your own file for testing.**

**-- Have attached the randomizer below**

In [3]:
## Install if needed.
pip install yfinance pandas sqlalchemy mysql-connector-python

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.6 MB ? eta -:--:--
   ------------------- -------------------- 0.8/1.6 MB 2.0 MB/s eta 0:00:01
   ------------------------- -------------- 1.0/1.6 MB 2.2 MB/s eta 0:00:01
   -------------------------------- ------- 1.3/1.6 MB 1.9 MB/s eta 0:00:01
   -------------------------------- ------- 1.3/1.6 MB 1.9 MB/s eta 0:00:01
   ---------------------------------------- 1.6/1.6 MB 1.3 MB/s  0:00:01
   ---------------------------------------- 0.0/16.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.5 MB ? eta -:--:--
   ------------

In [3]:
import pandas as pd
import yfinance as yf
import time

## Importing our stcok list data
df = pd.read_csv(r"C:\Users\ilaky\Documents\DataScience\AI-Driven_Multi-Asset-Intelligence_&_Portfolio-Engine\data\ind_nifty500list.csv") ## Remap ur file

## Formating stock ticker to yfinance approved format [adding .NS]
df["Ticker"] = df["Symbol"] + ".NS"

## Fetching each Stocks MarketCap
def stock_data_fetcher(ticker_list):
    fetched_stock_data = []
    print(f"Fetching the data for {len(ticker_list)} stocks")
    for i,ticker in enumerate(ticker_list):
        try:
            stock= yf.Ticker(ticker)
            info = stock.info
            fetched_stock_data.append({
                "Ticker":ticker,
                "Market_Cap_value":info.get("marketCap"),
                "Sector":info.get("sector")}) ## If needed can add required field in the same format
            
        except Exception as ex:  ## If data is available
            print(f"Skipping {ticker} due to error.")
            fetched_stock_data.append({"Ticker":ticker,"Market_Cap_value":info.get(None),"Current_Price":info.get(None)})
            
        if i % 10 == 0:
            pass
            #print(f"Progress :{(i/len(ticker_list)) * 100} %")
        time.sleep(0.1)    ## To avoid being blocked by yfinance api.
    print(f"Progress :{(i/len(ticker_list)) * 100} % is done")
    return pd.DataFrame(fetched_stock_data)

## Running the function
ticker_list = df["Ticker"].head(500).tolist()
fetched_df = stock_data_fetcher(ticker_list)

## Merging the Dataframes
merged_df = pd.merge(df,fetched_df,on = "Ticker")
merged_df = merged_df.sort_values(by = "Market_Cap_value",ascending=False).drop(columns =["Series","ISIN Code","Symbol"]).reset_index(drop=True)

df_categorized = merged_df.copy()
df_categorized["Market_Cap_Cat"] = "Small" 
df_categorized.iloc[0:100,df_categorized.columns.get_loc("Market_Cap_Cat")] = "Large"
df_categorized.iloc[100:250,df_categorized.columns.get_loc("Market_Cap_Cat")] = "Mid"

Sample_df = df_categorized.copy()
Sample_df.rename(columns = {"Company Name":"Company_Name","Market_Cap_value":"Cap_value"},inplace=True)
col_order = ["Ticker","Company_Name","Sector","Industry","Market_Cap_Cat","Cap_value"]
Sample_df = Sample_df[col_order]

Sample_df.to_csv("fetched_stocks.csv",index=False)
Sample_df

Fetching the data for 500 stocks
Progress :99.8 % is done


,Ticker,Company_Name,Sector,Industry,Market_Cap_Cat,Cap_value
0,RELIANCE.NS,Reliance Industries Ltd.,Energy,Oil Gas & Consumable Fuels,Large,19635617398784
1,HDFCBANK.NS,HDFC Bank Ltd.,Financial Services,Financial Services,Large,14480122576896
2,BHARTIARTL.NS,Bharti Airtel Ltd.,Communication Services,Telecommunication,Large,12323047604224
3,TCS.NS,Tata Consultancy Services Ltd.,Technology,Information Technology,Large,10638262796288
4,ICICIBANK.NS,ICICI Bank Ltd.,Financial Services,Financial Services,Large,10071887052800
...,...,...,...,...,...,...
495,SAREGAMA.NS,Saregama India Ltd,Communication Services,Media Entertainment & Publication,Small,63011438592
496,TEJASNET.NS,Tejas Networks Ltd.,Technology,Telecommunication,Small,59484917760
497,HAPPSTMNDS.NS,Happiest Minds Technologies Ltd.,Technology,Information Technology,Small,58274652160
498,PRAJIND.NS,Praj Industries Ltd.,Industrials,Capital Goods,Small,52690022400


----

In [19]:
Sample_df.isnull().sum()

Ticker              0
Company_Name        0
Sector              0
Industry            0
Market_Cap          0
Market_Cap_value    0
Current_Price       0
dtype: int64

In [20]:
Sample_df["Industry"].value_counts()

Industry
Financial Services                   95
Capital Goods                        64
Healthcare                           52
Automobile and Auto Components       35
Fast Moving Consumer Goods           31
Information Technology               28
Consumer Services                    26
Chemicals                            26
Oil Gas & Consumable Fuels           19
Consumer Durables                    19
Power                                18
Metals & Mining                      17
Services                             13
Construction                         12
Construction Materials               11
Telecommunication                    10
Realty                               10
Textiles                              6
Media Entertainment & Publication     4
Diversified                           3
Forest Materials                      1
Name: count, dtype: int64

In [21]:
Sample_df["Industry"].nunique()

21

In [9]:
st = yf.Ticker("RELIANCE.NS")

In [16]:
st.info

{'address1': 'Maker Chambers IV',
 'address2': '3rd Floor 222 Nariman Point',
 'city': 'Mumbai',
 'zip': '400021',
 'country': 'India',
 'phone': '91 22 3555 5000',
 'fax': '91 22 2204 2268',
 'website': 'https://www.ril.com',
 'industry': 'Oil & Gas Refining & Marketing',
 'industryKey': 'oil-gas-refining-marketing',
 'industryDisp': 'Oil & Gas Refining & Marketing',
 'sector': 'Energy',
 'sectorKey': 'energy',
 'sectorDisp': 'Energy',
 'longBusinessSummary': 'Reliance Industries Limited engages in the hydrocarbon exploration and production, oil and chemicals, retail, and digital service businesses worldwide. It operates through Oil to Chemicals, Oil and Gas, Retail, Digital Services, and Others segments. The company offers refining and marketing products, including liquefied petroleum gas, propylene, naphtha, gasoline, jet/aviation turbine fuel, kerosine oil, diesel, sulphur, and petroleum coke. It also provides polymers, including high-density and low-density polyethylene (PE), line

In [11]:
st.info.get("sector")

'Energy'

In [12]:
st.info.get("industry")

'Oil & Gas Refining & Marketing'

In [13]:
df.head()

,Company Name,Industry,Symbol,Series,ISIN Code,Ticker
0,360 ONE WAM Ltd.,Financial Services,360ONE,EQ,INE466L01038,360ONE.NS
1,3M India Ltd.,Diversified,3MINDIA,EQ,INE470A01017,3MINDIA.NS
2,ABB India Ltd.,Capital Goods,ABB,EQ,INE117A01022,ABB.NS
3,ACC Ltd.,Construction Materials,ACC,EQ,INE012A01025,ACC.NS
4,ACME Solar Holdings Ltd.,Power,ACMESOLAR,EQ,INE622W01025,ACMESOLAR.NS


In [15]:
df["Company Name"]

0                        360 ONE WAM Ltd.
1                           3M India Ltd.
2                          ABB India Ltd.
3                                ACC Ltd.
4                ACME Solar Holdings Ltd.
                      ...                
496    Zee Entertainment Enterprises Ltd.
497                 Zen Technologies Ltd.
498               Zensar Technolgies Ltd.
499               Zydus Lifesciences Ltd.
500                  eClerx Services Ltd.
Name: Company Name, Length: 501, dtype: str